# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

In [ ]:
print(y)

### Encoding categorical data

Label Encoding the "Gender" column

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X[:, 2])

One Hot Encoding the "Geography" column

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X[:, 1])

### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [12]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [16]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [ ]:
history = ann.fit(X_train, y_train, validation_split = 0.1, batch_size = 32, epochs = 100)

# Visualization of the Loss of training

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Epoch Number')
plt.ylabel("Loss")
plt.plot(history.history['loss'])

# Visualization of the Accuracy of training

In [ ]:
plt.xlabel('Epoch Number')
plt.ylabel("Accuracy Magnitude")
plt.plot(history.history['accuracy'])

# Visualization of the Loss of validation VS loss of training

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Visualization of the Accuracy of validation VS Accuracy of training

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Part 4 - Making the predictions and evaluating the model

### Predicting the Test set results

In [111]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

## Model Performance Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n\n{cm}')

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(accuracy*100)}%')

In [ ]:
tp, fp, fn, tn = cm.ravel()
TPR = tp/(tp+fn)
TNR = tn/(tn+fp)
print(f'TPR: {round(TPR*100)}%\n'
      f'TNR: {round(TNR*100)}%')

In [ ]:
from sklearn.metrics import classification_report
target_names = ['stay', 'leave']
print(classification_report(y_test, y_pred, target_names=target_names))

## ROC
The ROC curve helps us find the threshold where the TPR is high and FPR is low i.e. misclassifications are low. Therefore, ROC curves should be used to determine the optimal probability threshold for a classification model.

In [ ]:
from sklearn.metrics import roc_curve, auc

y_pred_ravel = ann.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_ravel)
auc_keras = auc(fpr_keras, tpr_keras)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred_ravel)

### Predicting the result of a single observation

**Solution**

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [25]:
def single_predictor(single_features):
    features_list = []
    for sublist in single_features.values():
        if type(sublist) == list:
            for item in sublist:
                features_list.append(item)
        else:
            features_list.append(sublist)
    transform_features = sc.transform([features_list])
    my_prediction = ann.predict(transform_features)
    print(my_prediction > 0.5)

In [26]:
single_featurs = {
    'Geography': [1, 0, 0] ,
    'Credit Score': 600,
    'Gender': 1,
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'Number of Products': 2,
    'Credit Card': 1,
    'Active Member': 1,
    'Salary': 50000,
}

In [ ]:
# print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
single_predictor(single_featurs)